In [1]:
import numpy as np

import random
from collections import deque
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal

import cv2

import game

from IPython.display import clear_output

ModuleNotFoundError: No module named 'torch'

In [156]:
from importlib import reload
reload(game)

<module 'game' from '/m/home/home7/70/viitala1/data/Documents/Dippa/driving_simulator/game.py'>

In [1]:
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pygame.display.quit()

In [157]:
im_cols = im_rows = 80
state_size = np.array((im_cols, im_rows))
action_space = np.array((2,1))
img_channels = 1

act_size = 2

hidden_size = 512

batch_size = 64
lr = 0.0001

n_random_episodes = 10

env = game.game(True)

In [158]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [159]:
class Conv(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, 24, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(24, 32, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            Flatten(),
        )
    def forward(self, x):
        #print(x.shape)
        #inp = x.reshape((64, 1, 80, 80))
        #inp = x.unsqueeze(0)
        return self.net(x)

In [160]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size, init_w=3e-3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 512), nn.ReLU(), 
            nn.Linear(512, output_size)
        )
        self.net[-1].weight.data.uniform_(-init_w, init_w)
        self.net[-1].bias.data.uniform_(-init_w, init_w)

    def forward(self, x):
        return self.net(x)

In [161]:
class Critic(nn.Module):
    """ Twin Q-networks """
    def __init__(self):
        super().__init__()
        self.net1 = MLP(6402, 1)
        self.net2 = MLP(6402, 1)

    def forward(self, state, action):
        state_action = torch.cat([state, action], 1)
        return self.net1(state_action), self.net2(state_action)

In [162]:
class Actor(nn.Module):
    """ Gaussian Policy """
    def __init__(self):
        super().__init__()
        self.net = MLP(6400, 2*2)

    def forward(self, state):
        x = self.net(state)
        mean, log_std = x[:, :act_size], x[:, act_size:]
        log_std = torch.clamp(log_std, min=-20, max=2)
        return mean, log_std

    def sample(self, state):
        
        mean, log_std = self.forward(state)
        
        normal = Normal(mean, log_std.exp())
        x = normal.rsample()

        # Enforcing action bounds
        action = torch.tanh(x)
        log_prob = normal.log_prob(x) - torch.log(1 - action**2 + 1e-6)
        log_prob = log_prob.sum(1, keepdim=True)

        return action, log_prob

    def select_action(self, state):
        state = torch.FloatTensor(state).to(device).unsqueeze(0)
        action, _ = self.sample(conv.forward(state))
        return action[0].detach().cpu().numpy()

In [163]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#device = torch.device("cpu")

conv = Conv().to(device)
conv_optimzer = torch.optim.Adam(conv.parameters(), lr=lr)

critic = Critic().to(device)
critic_optimizer = torch.optim.Adam(critic.parameters(), lr=lr)

critic_target = Critic().to(device)
for target_param, param in zip(critic_target.parameters(), critic.parameters()):
    target_param.data.copy_(param.data)
    
actor = Actor().to(device)
actor_optimizer = torch.optim.Adam(actor.parameters(), lr=lr)

target_entropy = -act_size
log_alpha = torch.zeros(1, requires_grad=True, device=device)
alpha_optimizer = torch.optim.Adam([log_alpha], lr=lr)

In [164]:
def update_parameters(replay_buffer):
    batch = random.sample(replay_buffer, k=batch_size)
    state, action, reward, next_state, not_done = [torch.FloatTensor(t).to(device) for t in zip(*batch)]

    alpha = log_alpha.exp().item()

    # Update critic_action1
    
    
    
    with torch.no_grad():
        embedding = conv.forward(next_state.reshape(batch_size, 1, 80, 80))
        next_action, next_action_log_prob = actor.sample(embedding)
        q1_next, q2_next = critic_target(embedding, next_action)
        q_next = torch.min(q1_next, q2_next)
        value_next = q_next - alpha * next_action_log_prob
        q_target = reward + not_done * 0.0001 * value_next
    
    prev_embedding = conv.forward(state.reshape(batch_size, 1, 80, 80))
    
    q1, q2 = critic(prev_embedding, action)
    q1_loss = 0.5*F.mse_loss(q1, q_target)
    q2_loss = 0.5*F.mse_loss(q2, q_target)
    critic_loss = q1_loss + q2_loss

    critic_optimizer.zero_grad()
    critic_loss.backward(retain_graph=True)
    critic_optimizer.step()

    for target_param, param in zip(critic_target.parameters(), critic.parameters()):
        target_param.data.copy_((1.0-0.001)*target_param.data + 0.001*param.data)

    # Update actor

    action_new, action_new_log_prob = actor.sample(embedding)
    q1_new, q2_new = critic(prev_embedding, action_new)
    q_new = torch.min(q1_new, q2_new)
    actor_loss = (alpha*action_new_log_prob - q_new).mean()

    actor_optimizer.zero_grad()
    actor_loss.backward(retain_graph=True)
    actor_optimizer.step()

    # Update alpha

    alpha_loss = -(log_alpha * (action_new_log_prob + target_entropy).detach()).mean()

    alpha_optimizer.zero_grad()
    alpha_loss.backward(retain_graph=True)
    alpha_optimizer.step()

In [6]:
env

{'CLUTTER_IM_MODULE': 'xim',
 'CONDA_SHLVL': '1',
 'LD_LIBRARY_PATH': ':/home/ari/Documents/opt/gurobi810/linux64/lib',
 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.wim=01;31:*.swm=01;31:*.dwm=01;31:*.esd=01;31:*.jpg=01;35:*.jpeg=01;35:*.mjpg=01;35:*.mjpeg=01;35:*.gif=01;35:*.bmp=01;35:*.pbm=01;35:*.pgm=01;35:*.ppm=01;35:*.tga=01;35:*.xbm=01;35:*.xpm=01;35:

In [165]:
def process_image(obs):

        result = cv2.resize(obs, (im_rows, im_cols))

        return result

In [167]:
replay_buffer = deque(maxlen=10000)

for episode in range(10000):
    state = process_image(env.reset())
    episode_reward = 0
    
    done = False
    while not done:
        if episode < n_random_episodes:
            action = np.random.rand(2) - 0.5
        else:
            action = actor.select_action(state.reshape(1, 80, 80))

        next_state, reward, done = env.step(action[0], 1, np.random.rand(80, 80))
        next_state = process_image(next_state)
        episode_reward += reward

        not_done = not done
        replay_buffer.append([state, action, [reward], next_state, [not_done]])
        state = next_state

        if len(replay_buffer) > batch_size:
            update_parameters(replay_buffer)
    
    #clear_output(wait = True)
    print("Episode {}. Reward {}".format(episode, episode_reward))
    

Episode 0. Reward 48.90264377821428
Episode 1. Reward 25.08904887232758
Episode 2. Reward 25.08904887232758
Episode 3. Reward 100.88893046049103
Episode 4. Reward 40.922470161655966
Episode 5. Reward 100.88893046049103
Episode 6. Reward 96.88895384451722
Episode 7. Reward 40.922470161655966
Episode 8. Reward 104.88891549471425
Episode 9. Reward 56.894522891556576
Episode 10. Reward 21.201638863011844
Episode 11. Reward 21.201638863011844
Episode 12. Reward 25.08904887232758
Episode 13. Reward 25.08904887232758
Episode 14. Reward 25.08904887232758
Episode 15. Reward 21.201638863011844
Episode 16. Reward 25.08904887232758
Episode 17. Reward 25.08904887232758
Episode 18. Reward 21.201638863011844
Episode 19. Reward 25.08904887232758
Episode 20. Reward 25.08904887232758
Episode 21. Reward 36.941359627587445
Episode 22. Reward 25.08904887232758
Episode 23. Reward 104.88891549471425
Episode 24. Reward 25.08904887232758
Episode 25. Reward 52.89769201805714
Episode 26. Reward 56.89452289155657

In [ ]:
a = torch.tensor(np.random.rand(62, 1, 2))

In [ ]:
b = torch.tensor(np.random.rand(62, 1, 200))

In [154]:
b.shape

torch.Size([62, 1, 200])

In [160]:
a.T.shape

torch.Size([2, 1, 62])

In [161]:
b.T.shape

torch.Size([200, 1, 62])

In [158]:
a.shape

torch.Size([62, 1, 2])

In [162]:
torch.cat((a.T, b.T)).shape

torch.Size([202, 1, 62])

In [ ]:
class agent:
    def __init__(self, state_size, action_space):
        
        self.state_size = state_size
        self.action_space = action_space
        
        self.model = self.build_model()
        
        self.learning_rate = 0.0001
        
        def build_model(self):
            model = Sequential()
            model.add(Conv2D(24, (5, 5), strides=(2, 2), padding="same",input_shape=(img_rows,img_cols,img_channels), activation="relu"))  #80*80*4
            model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same", activation="relu"))
            model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same", activation="relu"))
            model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="relu"))
            model.add(Conv2D(64, (3, 3), strides=(1, 1), padding="same", activation="relu"))
            
            model.add(Flatten())
            
            model.add(Dense(hidden_size, activation="relu"))
            model.add(Dense(hidden_size, activation="relu"))
            model.add(Dense(hidden_size, activation="relu"))
            model.add(Dense(len(self.action_space), activation="linear"))
            
            adam = Adam(lr=self.learning_rate)
            model.compile(optimizer=adam)
            
        
            

In [29]:
np.random.rand?

Docstring:
rand(d0, d1, ..., dn)

Random values in a given shape.

Create an array of the given shape and populate it with
random samples from a uniform distribution
over ``[0, 1)``.

Parameters
----------
d0, d1, ..., dn : int, optional
    The dimensions of the returned array, should all be positive.
    If no argument is given a single Python float is returned.

Returns
-------
out : ndarray, shape ``(d0, d1, ..., dn)``
    Random values.

See Also
--------
random

Notes
-----
This is a convenience function. If you want an interface that
takes a shape-tuple as the first argument, refer to
np.random.random_sample .

Examples
--------
>>> np.random.rand(3,2)
array([[ 0.14022471,  0.96360618],  #random
       [ 0.37601032,  0.25528411],  #random
       [ 0.49313049,  0.94909878]]) #random
Type:      builtin_function_or_method


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size, init_w=3e-3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, 24, 5, stide=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(24, 32, 5, stide=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 5, stide=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stide=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stide=1, padding=1),
            nn.ReLU(),
            Flatten(),
            nn.Linear(input_size, args.hidden_size), nn.ReLU(),
            nn.Linear(args.hidden_size, args.hidden_size), nn.ReLU(),
            nn.Linear(args.hidden_size, args.hidden_size), nn.ReLU(),
            nn.Linear(args.hidden_size, output_size)
        )
        self.net[-1].weight.data.uniform_(-init_w, init_w)
        self.net[-1].bias.data.uniform_(-init_w, init_w)

    def forward(self, x):
        return self.net(x)

In [13]:
Sequential.compile?

Signature:
Sequential.compile(
    self,
    optimizer,
    loss=None,
    metrics=None,
    loss_weights=None,
    sample_weight_mode=None,
    weighted_metrics=None,
    target_tensors=None,
    distribute=None,
    **kwargs,
)
Docstring:
Configures the model for training.

Arguments:
    optimizer: String (name of optimizer) or optimizer instance.
        See `tf.keras.optimizers`.
    loss: String (name of objective function) or objective function.
        See `tf.losses`. If the model has multiple outputs, you can use a
        different loss on each output by passing a dictionary or a list of
        losses. The loss value that will be minimized by the model
        will then be the sum of all individual losses.
    metrics: List of metrics to be evaluated by the model
        during training and testing.
        Typically you will use `metrics=['accuracy']`.
        To specify different metrics for different outputs of a
        multi-output model, you could also pass a dictiona